In [1]:
%%init_spark
launcher.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")
launcher.packages = ["org.twitter4j:twitter4j-core:4.0.7"]

# Carrega os tweets e converte em json

In [ ]:
import twitter4j.{Twitter, TwitterFactory}
import twitter4j.Query
import scala.collection._
import scala.collection.JavaConverters

val twitter = TwitterFactory.getSingleton()
val query = new Query("#convid19")
val tweets = twitter.search(query).getTweets()
val result = JavaConverters.collectionAsScalaIterableConverter(tweets).asScala.toSeq
val jsonTweets = result.map(x => TwitterObjectFactory.getRawJSON(x))

# Salva os tweets particionados por data como parquet

In [ ]:
import org.apache.spark.sql.{DataFrame, Row, SQLContext, DataFrameReader}
import org.apache.spark.sql.functions._
import spark.implicits._
import twitter4j.TwitterObjectFactory

val df = spark.read.json(jsonTweets.toDS)
    .withColumn("dt", lit("2020-10-25"))

df.write
    .partitionBy("dt")
    .mode("overwrite")
    .parquet("/data/tweets")